In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Feature Store Based LLM Grounding tutorial

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_based_llm_grounding_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ffeature_store%2Fvertex_ai_feature_store_based_llm_grounding_tutorial.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/vertex_ai_feature_store_based_llm_grounding_tutorial.ipynb.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_based_llm_grounding_tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td>
</table>

## Overview

In this tutorial, you learn how to chunk user-provided data, and then generate embedding vectors for each chunk using a Vertex LLM (Large Language Model) having embedding generation capabilities. The resulting embedding vector dataset can then be loaded into Vertex AI Feature Store, enabling fast feature retrieval and efficient online serving.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/overview).

### Objective

In this tutorial, you learn how to create and use an online feature store instance to host and serve data in BigQuery with Vertex AI Feature Store in an end to end workflow of features serving and vector retrieval user journey.

This tutorial uses the following Google Vertex AI services and resources:

- Vertex AI Feature Store

The steps performed include:

- Provision an online feature store instance to host and serve data.
- Create an online feature store instance to serve a BigQuery table.
- Use the online server to search nearest neighbors.

### Note
This is a Preview release. By using the feature, you acknowledge that you're aware of the open issues and that this preview is provided “as is” under the pre-GA terms of service.


### Dataset

This tutorial uses the [Google Patents Public Data](https://console.cloud.google.com/marketplace/product/google_patents_public_datasets/google-patents-public-data) dataset from the BigQuery public datasets.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[BigQuery pricing](https://cloud.google.com/bigquery/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform\
                                 google-cloud-bigquery\
                                 db-dtypes

! pip3 install --upgrade kfp -q --no-warn-conflicts

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>

### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"

### Import libraries

In [ ]:
import uuid

from google.cloud import bigquery
from google.cloud.aiplatform_v1 import (FeatureOnlineStoreAdminServiceClient,
                                        FeatureOnlineStoreServiceClient)
from google.cloud.aiplatform_v1.types import NearestNeighborQuery
from google.cloud.aiplatform_v1.types import \
    feature_online_store as feature_online_store_pb2
from google.cloud.aiplatform_v1.types import \
    feature_online_store_admin_service as \
    feature_online_store_admin_service_pb2
from google.cloud.aiplatform_v1.types import \
    feature_online_store_service as feature_online_store_service_pb2

## Set up and start online serving

To serve embedding data in Vertex AI Feature Store, do the following:

1. Prepare the data source in BigQuery.
2. Create an FeatureOnlineStore instance to host the data.
3. Define the data (`FeatureView`) to be served by the newly-created instance.

### Prepare BigQuery data source for feature view creation

In [ ]:
GCS_BUCKET = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {GCS_BUCKET}

#### Prepare data in Google Cloud Storage (GCS)

In [ ]:
INPUT_TEXT_GCS_DIR = f"{GCS_BUCKET}/fs_grounding/data"

import tarfile
from urllib.request import urlretrieve


def untar(file_name):
    output_folder_name = file_name[:-7]
    file = tarfile.open(file_name)
    file.extractall(output_folder_name)
    return output_folder_name


# Download data from https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz
url = "https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/nfcorpus.tar.gz"
filename = "nfcorpus.tar.gz"
path, _ = urlretrieve(url, filename)
print(f"Downloaded {path}")

# Copy text files to GCS.
output_folder_name = f"{untar(path)}/nfcorpus"
dev_all_queries = f"{output_folder_name}/dev.all.queries"
dev_docs = f"{output_folder_name}/dev.docs"
! gsutil cp {dev_all_queries} {INPUT_TEXT_GCS_DIR}/queries
! gsutil cp {dev_docs} {INPUT_TEXT_GCS_DIR}/docs

#### Create BigQuery dataset

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

BQ_DATASET_ID = "fs_grounding"  # @param {type:"string"}
dataset = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET_ID}")
dataset.location = LOCATION
dataset = bq_client.create_dataset(
    dataset, exists_ok=True, timeout=30
)  # Make an API request.

# Confirm dataset created.
print(f"Created dataset {dataset}.{BQ_DATASET_ID}")

#### Launch pipeline

In [ ]:
run_id = str(uuid.uuid4())

PIPELINE_TEMPLATE_URI = "gs://vertex-evaluation-pipeline-templates/20240117_0005/feature_store_grounding_pipeline_pipeline.yaml"
BIGQUERY_BP_INPUT_URI = f"bq://{PROJECT_ID}.{BQ_DATASET_ID}.batch_predict_input"
BIGQUERY_BP_OUTPUT_URI = f"bq://{PROJECT_ID}.{BQ_DATASET_ID}.batch_predict_output"

PARAMS = {
    "project": PROJECT_ID,
    "location": LOCATION,
    "bigquery_bp_input_uri": BIGQUERY_BP_INPUT_URI,
    "bigquery_bp_output_uri": BIGQUERY_BP_OUTPUT_URI,
    "input_text_gcs_dir": INPUT_TEXT_GCS_DIR,
    "output_text_gcs_dir": f"{GCS_BUCKET}/fs_grounding_{run_id}/chunking_output",
    "output_error_file_path": f"{GCS_BUCKET}/fs_grounding_{run_id}/chunking_error_output",
    "model_name": "publishers/google/models/textembedding-gecko@latest",
    "generation_threshold_microseconds": "0",
}


def run_pipeline(
    parameters: dict,
    project: str,
    pipeline_root: str,
    location: str = "us-central1",
) -> aiplatform.PipelineJob:
    aiplatform.init(
        project=project,
        location=location,
    )

    test_prefix = "your-test-prefix"  # @param {type:"string"}
    pipeline_name = "feature-store-grounding-pipeline"  # @param {type:"string"}

    test_name = f"{test_prefix}-{pipeline_name}-{run_id}"
    job = aiplatform.PipelineJob(
        display_name=test_name,
        template_path=PIPELINE_TEMPLATE_URI,
        job_id=test_name,
        pipeline_root=pipeline_root,
        parameter_values=parameters,
        enable_caching=False,
    )

    job.submit()

    return job


job = run_pipeline(
    parameters=PARAMS,
    project=PROJECT_ID,
    pipeline_root=f"{GCS_BUCKET}/fs_based/pipeline_root",
    location=LOCATION,
)
job.wait()

#### BQ format conversion

In [ ]:
def compose_bq_query_format_conversion(
    bigquery_bp_input_uri: str, bigquery_bp_output_uri: str
) -> str:
    """Compose the BQ query for format conversion.

    Args:
      bigquery_bp_input_uri: The URI to a bigquery table as the input for the
        batch prediction component. The chunking component populates data to
        this uri first before batch prediction.
      bigquery_bp_output_uri: The URI to a bigquery table as the output for the
        batch prediction component.

    Returns:
      The composed query for BigQuery format conversion.
    """

    if bigquery_bp_input_uri.startswith("bq://"):
        bigquery_bp_input_uri = bigquery_bp_input_uri.replace("bq://", "")

    if bigquery_bp_output_uri.startswith("bq://"):
        bigquery_bp_output_uri = bigquery_bp_output_uri.replace("bq://", "")

    inseration_query = (
        f"UPDATE `{bigquery_bp_input_uri}` destTable"
        " SET embedding=ARRAY( select cast (str_element as float64) from"
        " unnest(JSON_VALUE_ARRAY(prediction, '$.embeddings.values')) as"
        " str_element)"
    )
    fetch_data_query = (
        "FROM (SELECT vertex_generated_chunk_id, prediction FROM"
        f" `{bigquery_bp_output_uri}` cross join"
        " unnest(JSON_EXTRACT_ARRAY(predictions)) as prediction) sourceTable"
        " WHERE"
        " destTable.vertex_generated_chunk_id=sourceTable.vertex_generated_chunk_id"
    )
    return f"{inseration_query} {fetch_data_query};"


bq_query = compose_bq_query_format_conversion(
    bigquery_bp_input_uri=BIGQUERY_BP_INPUT_URI,
    bigquery_bp_output_uri=BIGQUERY_BP_OUTPUT_URI,
)

bq_job = bq_client.query(bq_query)
bq_job.result()

### Initialize Admin Service Client

Load the Feature Store SDK.

In [ ]:
admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

### Create Feature Online Store

Create a feature online store with embedding management enabled.

In [ ]:
FEATURE_ONLINE_STORE_ID = "my_feature_online_store_unique"  # @param {type: "string"}

In [ ]:
online_store_config = feature_online_store_pb2.FeatureOnlineStore(
    optimized=feature_online_store_pb2.FeatureOnlineStore.Optimized(),
)

create_store_lro = admin_client.create_feature_online_store(
    feature_online_store_admin_service_pb2.CreateFeatureOnlineStoreRequest(
        parent=f"projects/{PROJECT_ID}/locations/{LOCATION}",
        feature_online_store_id=FEATURE_ONLINE_STORE_ID,
        feature_online_store=online_store_config,
    )
)

### Verify online store instance creation

After the long-running operation (LRO) is complete, show the result.

> **Note:** This operation might take up to 10 minutes to complete.

In [ ]:
# Wait for the LRO to finish and get the LRO result.
print(create_store_lro.result())

#### Verify `FeatureOnlineStore` instance creation by retrieving the online store instance

In [ ]:
# Use get to verify the store is created.
admin_client.get_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)

#### List all online stores for the location

In [ ]:
# Use list to verify the store is created.
admin_client.list_feature_online_stores(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}"
)

### Create feature view instance

After creating a `FeatureOnlineStore` instance, you can define the features to serve with it. To do this, create a `FeatureView` instance, which specifies the following:

* A data source (BigQuery table or view URI or `FeatureGroup/features`) synced to the `FeatureOnlineStore` instance for serving.
* The [cron](https://en.wikipedia.org/wiki/Cron) schedule to run the sync pipeline.

During feature view creation, a sync job is scheduled, and either started immediately or following the cron schedule. In the sync job, data is exported, an index is built and deployed to GKE cluster.

In [ ]:
FEATURE_VIEW_ID = "fs_grounding_test_new"  # @param {type: "string"}
# A schedule is created based on cron setting.
# If cron is unspecified, a sync job is started immediately.
CRON_SCHEDULE = "TZ=America/Los_Angeles 00 13 11 8 *"  # @param {type: "string"}

In [ ]:
# Index building configs
DIMENSIONS = 768  # @param {type: "number"}
EMBEDDING_COLUMN = "embedding"  # @param {type: "string"}
# Optional
LEAF_NODE_EMBEDDING_COUNT = 10000  # @param {type: "number"}
# Optional
# CROWDING_COLUMN = "cited_by_filing_date"  # @param {type: "string"}
# # Optional
# FILTER_COLUMNS = ["country"]  # @param

In [ ]:
DATA_SOURCE = BIGQUERY_BP_INPUT_URI

In [ ]:
from google.cloud.aiplatform_v1.types import feature_view as feature_view_pb2

big_query_source = feature_view_pb2.FeatureView.BigQuerySource(
    uri=DATA_SOURCE, entity_id_columns=["vertex_generated_chunk_id"]
)

sync_config = feature_view_pb2.FeatureView.SyncConfig(cron=CRON_SCHEDULE)

index_config = feature_view_pb2.FeatureView.IndexConfig(
    embedding_column=EMBEDDING_COLUMN,
    # filter_columns=FILTER_COLUMNS,
    # crowding_column=CROWDING_COLUMN,
    embedding_dimension=DIMENSIONS,
    tree_ah_config=feature_view_pb2.FeatureView.IndexConfig.TreeAHConfig(),
)

print(f"index_config: {index_config}")

create_view_lro = admin_client.create_feature_view(
    feature_online_store_admin_service_pb2.CreateFeatureViewRequest(
        parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}",
        feature_view_id=FEATURE_VIEW_ID,
        feature_view=feature_view_pb2.FeatureView(
            big_query_source=big_query_source,
            sync_config=sync_config,
            index_config=index_config,
        ),
    )
)

 Wait for LRO to complete and show result

In [ ]:
print(create_view_lro.result())

### Verify feature view creation

Verify `FeatureView` instance creation by retrieving the feature view.

In [ ]:
admin_client.get_feature_view(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

Verify that the `FeatureView` instance is created by listing all the feature views within the online store.

In [ ]:
admin_client.list_feature_views(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)

In [ ]:
# Optional: Delete feature views to avoid exceeding the deployed index nodes quota.
# views = admin_client.list_feature_views(
#     parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
# )
# for view in views:
#     admin_client.delete_feature_view(name=view.name)

### Feature view syncs

The sync pipeline executes according to the schedule specified in the `FeatureView` instance.

To skip the wait and execute the sync pipeline immediately, start the sync manually.

In [ ]:
sync_response = admin_client.sync_feature_view(
    feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

The `sync_response` contains the ID of the sync job.

#### Use `get_feature_view_sync` to check the status of the job

In [ ]:
import time

while True:
    feature_view_sync = admin_client.get_feature_view_sync(
        name=sync_response.feature_view_sync
    )
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

#### Use `list_feature_view_syncs` to view all your syncs

In [ ]:
admin_client.list_feature_view_syncs(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

### Start online serving

After the data sync is complete, use the `FetchFeatureValuesRequest` and `SearchNearestEntities` APIs to retrieve the public endpoint domain name.

Get public endpoint domain name.

In [ ]:
# Verify online store creation.
featore_online_store_instance = admin_client.get_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)
PUBLIC_ENDPOINT = (
    featore_online_store_instance.dedicated_serving_endpoint.public_endpoint_domain_name
)

print(f"PUBLIC_ENDPOINT for online serving: {PUBLIC_ENDPOINT}")

#### Initialize the data client

In [ ]:
# It will take some time for the DNS to be fully ready
time.sleep(300)

data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": PUBLIC_ENDPOINT}
)

#### Search with `ENTITY_ID`

In [ ]:
bq_query = f'SELECT * FROM `{BIGQUERY_BP_INPUT_URI.replace("bq://", "")}` LIMIT 1'

bq_query_job = bq_client.query(bq_query)
result = bq_query_job.result().to_dataframe()

print(result)

ENTITY_ID = result["vertex_generated_chunk_id"][0]

In [ ]:
# A vertex_generated_chunk_id for testing
data_client.search_nearest_entities(
    request=feature_online_store_service_pb2.SearchNearestEntitiesRequest(
        feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}",
        query=NearestNeighborQuery(
            entity_id=ENTITY_ID,
            neighbor_count=5,
        ),
        return_full_entity=True,  # returning entities with metadata
    )
)

#### Search with `Embedding`

In [ ]:
EMBEDDINGS = [1] * DIMENSIONS

In [ ]:
data_client.search_nearest_entities(
    request=feature_online_store_service_pb2.SearchNearestEntitiesRequest(
        feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}",
        query=NearestNeighborQuery(
            embedding=NearestNeighborQuery.Embedding(value=EMBEDDINGS),
            neighbor_count=10,
        ),
        return_full_entity=True,  # returning entities with metadata
    )
)

#### Use the `FetchFeatureValues` API to retrieve the full data without search


In [ ]:
data_client.fetch_feature_values(
    request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
        feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}",
        data_key=feature_online_store_service_pb2.FeatureViewDataKey(key=ENTITY_ID),
    )
)

## Cleaning up

To clean up all Google Cloud resources used in this project, [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, delete the individual resources you created in this tutorial.

In [ ]:
# Delete Feature View
admin_client.delete_feature_view(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

# Delete Feature Online Store
admin_client.delete_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}",
    force=True,
)

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket:
    ! gsutil -m rm -r $GCS_BUCKET